In [1]:
# import required packages

import pandas as pd
import numpy as np

## Load Files

In [2]:
# load CSV of championship results into a dataframe and display first 10 records

championship_df = pd.read_csv("resources/yh-championship-data.csv")
championship_df.head(10)

,Year,Division,Horse,OverallPlacing,HighestLevel,CDI,USDFNumber,OverallScore
0,2002,FEI5,Rosabella,1,Third Level,No,37232,7.84
1,2002,FEI5,Favereux,2,Grand Prix,No,38714,7.68
2,2002,FEI5,Devon,3,Third Level,No,38984,7.42
3,2002,FEI5,Welfenstein,4,Grand Prix,Yes,40474,7.14
4,2002,FEI5,R-tistik,5,Grand Prix,Yes,37123,7.10
5,2002,FEI5,Pampero,6,FEI 5 Year Old,No,41386,6.82
6,2002,FEI6,Oleander,1,Grand Prix,Yes,35062,8.24
7,2002,FEI6,Freestyle,2,Prix St. Georges,No,39380,7.40
8,2002,FEI6,Wincenzo,3,Prix St. Georges,No,1026740,7.00
9,2002,FEI6,Olympus,4,Grand Prix,Yes,42683,6.86


In [44]:
# load CSV of horse data into a dataframe and display first 10 results

horse_df = pd.read_csv("resources/yh-horse-data.csv")
horse_df.head(10)


,Horse,HighestLevel,CDI,USDFNumber,Sire,Damsire,Country,Breeder,Studbook,InternationalTeam,TeamMade
0,Rosabella,Third Level,No,37232,Rohdiamant,Watzmann,Germany,Kerstin Ohlemeyer,Hanoverian,No,NaN
1,Favereux,Grand Prix,No,38714,Fidermark,Fidelio,Germany,Johannes Hilgers,Rhinelander,No,NaN
2,Devon,Third Level,No,38984,Don Gregory,NaN,NaN,NaN,Oldenburg,No,NaN
3,Welfenstein,Grand Prix,Yes,40474,Wolkenstein II,Lauries Crusador xx,Germany,Heinz Bruns,Hanoverian,No,NaN
4,R-tistik,Grand Prix,Yes,37123,Ramires,Rex Fritz,Germany,Josef Kathmann,Oldenburg,No,NaN
5,Pampero,FEI 5 Year Old,No,41386,Ferro,NaN,USA,Margaret Avery,KWPN,No,NaN
6,Oleander,Grand Prix,Yes,35062,Jazz,Ulft,Netherlands,R. Van Wourdenbergh,KWPN,No,NaN
7,Freestyle,Prix St. Georges,No,39380,Florestan I,Parademarsch I,Germany,NaN,Westfalen,No,NaN
8,Wincenzo,Prix St. Georges,No,1026740,Werther,Graphit,Germany,NaN,Hanoverian,No,NaN
9,Olympus,Grand Prix,Yes,42683,Clavecimbel,NaN,Netherlands,G. Van Der Veen,KWPN,No,NaN


In [54]:
# create merged dataframe

merged_df = pd.merge(championship_df, horse_df, on='USDFNumber')
merged_df.head(10) 

,Year,Division,Horse_x,OverallPlacing,HighestLevel_x,CDI_x,USDFNumber,OverallScore,Horse_y,HighestLevel_y,CDI_y,Sire,Damsire,Country,Breeder,Studbook,InternationalTeam,TeamMade
0,2002,FEI5,Rosabella,1,Third Level,No,37232,7.84,Rosabella,Third Level,No,Rohdiamant,Watzmann,Germany,Kerstin Ohlemeyer,Hanoverian,No,NaN
1,2002,FEI5,Favereux,2,Grand Prix,No,38714,7.68,Favereux,Grand Prix,No,Fidermark,Fidelio,Germany,Johannes Hilgers,Rhinelander,No,NaN
2,2002,FEI5,Devon,3,Third Level,No,38984,7.42,Devon,Third Level,No,Don Gregory,NaN,NaN,NaN,Oldenburg,No,NaN
3,2002,FEI5,Welfenstein,4,Grand Prix,Yes,40474,7.14,Welfenstein,Grand Prix,Yes,Wolkenstein II,Lauries Crusador xx,Germany,Heinz Bruns,Hanoverian,No,NaN
4,2002,FEI5,R-tistik,5,Grand Prix,Yes,37123,7.10,R-tistik,Grand Prix,Yes,Ramires,Rex Fritz,Germany,Josef Kathmann,Oldenburg,No,NaN
5,2002,FEI5,Pampero,6,FEI 5 Year Old,No,41386,6.82,Pampero,FEI 5 Year Old,No,Ferro,NaN,USA,Margaret Avery,KWPN,No,NaN
6,2002,FEI6,Oleander,1,Grand Prix,Yes,35062,8.24,Oleander,Grand Prix,Yes,Jazz,Ulft,Netherlands,R. Van Wourdenbergh,KWPN,No,NaN
7,2002,FEI6,Freestyle,2,Prix St. Georges,No,39380,7.40,Freestyle,Prix St. Georges,No,Florestan I,Parademarsch I,Germany,NaN,Westfalen,No,NaN
8,2002,FEI6,Wincenzo,3,Prix St. Georges,No,1026740,7.00,Wincenzo,Prix St. Georges,No,Werther,Graphit,Germany,NaN,Hanoverian,No,NaN
9,2002,FEI6,Olympus,4,Grand Prix,Yes,42683,6.86,Olympus,Grand Prix,Yes,Clavecimbel,NaN,Netherlands,G. Van Der Veen,KWPN,No,NaN


In [4]:
# get count of null values by column for horse dataframe

horse_df.isnull().sum(axis = 0)

Horse            0
HighestLevel     0
CDI              0
USDFNumber       0
Sire             1
Damsire         18
Country          8
Breeder         47
Studbook         1
dtype: int64

In [32]:
# get count of null values by column for championship dataframe

championship_df.isnull().sum(axis = 0)

Year              0
Division          0
Horse             0
OverallPlacing    0
HighestLevel      0
CDI               0
USDFNumber        0
OverallScore      0
dtype: int64

## Analysis of Competitive Outcomes, 2002-2019

In [5]:
# get the total number of horses that competed in the 4/5/6 year old divisions from 2002-2019

total_horses = championship_df.loc[(championship_df["Year"] <= 2019) &
                             (championship_df["Division"].str.contains("USEF4|FEI5|FEI6"))]
total_horses = total_horses["USDFNumber"].nunique()

print(f"The total number of horses competing from 2002-2019 is {total_horses}.")

The total number of horses competing from 2002-2019 is 520.


In [6]:
# get the overall number of horses that have competed in at least one CDI (any level other than Young Horse divisions)
# from 2002-2019, that competed in the 4/5/6/ year old divisions during those years

cdi_horses = championship_df.loc[(championship_df["CDI"] == "Yes") & (championship_df["Year"] <= 2019) &
                             (championship_df["Division"].str.contains("USEF4|FEI5|FEI6"))]
cdi_horses = cdi_horses["USDFNumber"].nunique()


print(f"The number of CDI competitors is {cdi_horses}.")

The number of CDI competitors is 169.


In [8]:
# get the overall percentage of horses that competed in at least one CDI 

cdi_percentage = (cdi_horses / total_horses) * 100
cdi_percentage = round(cdi_percentage, 2)

print(f"The percentage of CDI competitors is {cdi_percentage}%")

The percentage of CDI competitors is 32.5%


In [9]:
# get the number of horses that competed to Grand Prix from 2002-2019, 
# that competed in the 4/5/6/ year old divisions during those years

grandprix_horses = championship_df.loc[(championship_df["HighestLevel"] == "Grand Prix") & (championship_df["Year"] <= 2019) &
                             (championship_df["Division"].str.contains("USEF4|FEI5|FEI6"))]
grandprix_horses = grandprix_horses["USDFNumber"].nunique()


print(f"The number of Grand Prix horses is {grandprix_horses}.")

The number of Grand Prix horses is 138.


In [10]:
# get the percentage of all horses that competed at Grand Prix, from 2002-2019,
# that competed in the 4/5/6 year old divisions during those years

grand_prix_percentage = (grandprix_horses / total_horses) * 100
grand_prix_percentage = round(grand_prix_percentage, 2)

print(f"The percentage of Grand Prix horses is {grand_prix_percentage}%.")

The percentage of Grand Prix horses is 26.54%.


In [41]:
# get the horses that have competed at FEI (Prix St. Georges and above) from 2002-2019

levels = ['Grand Prix', 'I-2', 'I-A', 'I-B', 'I-1', 'Prix St. Georges','FEI Junior']

fei_df = championship_df[(championship_df['HighestLevel'].isin(levels)) & (championship_df["Year"] <= 2019) &
                             (championship_df["Division"].str.contains("USEF4|FEI5|FEI6"))]
fei_df = fei_df["USDFNumber"].nunique()

print(f"The number of horses that competed at FEI is {fei_df}.")

The number of horses that competed at FEI is 363.


In [15]:
# get the percentage of FEI horses that competed at Grand Prix, from 2002-2019,
# that competed in the 4/5/6 year old divisions during those years

fei_percentage = (fei_df / total_horses) * 100
fei_percentage = round(fei_percentage, 2)
print(f"The percentage of horses that competed at FEI is {fei_percentage}%.")

The percentage of horses that competed at FEI is 69.81%.


In [51]:
# get a list of the horses that went on to make an international team (Pan American or Olympic Games)

team_df = merged_df[(merged_df["Year"] <= 2019) & (merged_df["InternationalTeam"] == "Yes") &
                             (merged_df["Division"].str.contains("USEF4|FEI5|FEI6"))]
team_df = team_df.drop_duplicates(subset='USDFNumber')

print(team_df)

     Year Division       Horse_x  OverallPlacing HighestLevel_x CDI_x  \
166  2007     FEI6     Grandioso               1            I-2   Yes   
238  2008    USEF4     Selten HW               1     Grand Prix   Yes   
399  2011     FEI5        Sanceo               4     Grand Prix   Yes   
633  2014    USEF4  Lucky Strike               4            I-1   Yes   

     USDFNumber  OverallScore       Horse_y HighestLevel_y CDI_y         Sire  \
166     1049928         8.368     Grandioso            I-2   Yes     Grosso Z   
238     1031805         8.560     Selten HW     Grand Prix   Yes   Sandro Hit   
399     1083924         8.116        Sanceo     Grand Prix   Yes     San Remo   
633     1106541         7.944  Lucky Strike            I-1   Yes  Lord Laurie   

             Damsire  Country                    Breeder    Studbook  \
166  Palisandergrund  Germany          Willi Hillebrecht   Westfalen   
238       Hohenstein      USA  Irene Hoeflich-Wiederhold  Hanoverian   
399  Ramiro'

In [16]:
# create summary table of overall horse level statistics

horse_summary = pd.DataFrame({"Total Horses": [total_horses], "Total FEI Horses": fei_df,
                            "Percentage of FEI Horses": fei_percentage, 
                            "Total CDI Horses": cdi_horses,
                            "Percentage of CDI Horses": cdi_percentage,
                            "Total Grand Prix Horses": grandprix_horses,
                            "Percentage of Grand Prix Horses": grand_prix_percentage})


horse_summary

,Total Horses,Total FEI Horses,Percentage of FEI Horses,Total CDI Horses,Percentage of CDI Horses,Total Grand Prix Horses,Percentage of Grand Prix Horses
0,520,363,69.81,169,32.5,138,26.54


In [380]:
# get the number of FEI horses that placed in the top 10 of their division at the championships between 2002-2019

placing = [1,2,3,4,5,6,7,8,9,10]
levels = ['Grand Prix', 'I-2', 'I-A', 'I-B', 'I-1', 'Prix St. Georges','FEI Junior']

top_ten = championship_df[(championship_df['OverallPlacing'].isin(placing)) & (championship_df['HighestLevel'].isin(levels))
                          & (championship_df["Year"] <= 2019) &
                             (championship_df["Division"].str.contains("USEF4|FEI5|FEI6"))]

top_ten = top_ten["USDFNumber"].nunique()

print(f"{top_ten} horses placed in the top 10 of their division.")




280 horses placed in the top 10 of their division.


In [381]:
# get the number of FEI horses that placed in the bottom 10 of their division between 2002-2019

lower_placing = [11,12,13,14,15,16,17,18,19,20]
levels = ['Grand Prix', 'I-2', 'I-A', 'I-B', 'I-1', 'Prix St. Georges','FEI Junior']

bottom_ten = championship_df[(championship_df['OverallPlacing'].isin(lower_placing)) & (championship_df['HighestLevel'].isin(levels))
                          & (championship_df["Year"] <= 2019) &
                             (championship_df["Division"].str.contains("USEF4|FEI5|FEI6"))]

bottom_ten = bottom_ten["USDFNumber"].nunique()

print(f"{bottom_ten} horses placed in the bottom 10 of their division.")

111 horses placed in the bottom 10 of their division.


In [382]:
# get the percentage of FEI horses in the top ten of their division

top_ten_percentage = (top_ten / fei_df) * 100
top_ten_percentage = round(top_ten_percentage, 2)

print(f"The percentage of horses placing in the top ten of their division is {top_ten_percentage}%.")

The percentage of horses placing in the top ten of their division is 77.13%.


In [383]:
# get the percentage of FEI horses in the top ten of their division

bottom_ten_percentage = (bottom_ten / fei_df) * 100
bottom_ten_percentage = round(bottom_ten_percentage, 2)

print(f"The percentage of horses placing in the top ten of their division is {bottom_ten_percentage}%.")

The percentage of horses placing in the top ten of their division is 30.58%.


## Bloodline and Breeding Data Analysis, 2002-2023

In [20]:
# get the median scores by division over all years (2002-2023)

median_groups = championship_df.groupby("Division")["OverallScore"]

# create a summary dataframe of median scores for all divisions over all years

median_summary = pd.DataFrame({"Median Score": median_groups.median()}) 

median_summary


,Median Score
Division,
DHGP,64.8880
DHPSG,67.3390
FEI5,7.5840
FEI6,7.5040
FEI7,71.3295
USEF4,7.6200


In [26]:
# get the number of horses bred in the USA that competed in the years 2002-2023

usa_count = horse_df[(horse_df["Country"] == "USA")].count()["USDFNumber"]

print(f"{usa_count} horses were bred in the USA.")

307 horses were bred in the USA.


In [29]:
# get the percentage of horses bred in the USA

all_horses = len(horse_df)
usa_percentage = (usa_count / all_horses) * 100
usa_percentage = round(usa_percentage, 2)

print(f"The percentage of horses bred in the USA is {usa_percentage}%.")

The percentage of horses bred in the USA is 36.5%


In [23]:
# get the top ten sires represented over all years (2002-2023)
# showing 11 horses, as there are three horses tied for the 9th place spot

top_sires = horse_df['Sire'].value_counts().head(11)

print(top_sires)


Sandro Hit          24
Sir Donnerhall I    18
Furstenball         17
Jazz                13
Rotspon             12
Fidertanz           12
Florestan I         12
Hotline             10
Sir Sinclair         9
Florencio I          9
Grand Galaxy Win     9
Name: Sire, dtype: int64

In [24]:
# get the top ten sires of Grand Prix horses from 2002-2023

gp_sires = horse_df.loc[(horse_df["HighestLevel"] == "Grand Prix")]
gp_sires = gp_sires['Sire'].value_counts().head(10)

print(gp_sires)

Sandro Hit          12
Jazz                 8
Sir Donnerhall I     7
Florestan I          5
Florencio I          5
Fidertanz            4
Quaterback           4
Rotspon              4
Belissimo M          4
Furstenball          3
Name: Sire, dtype: int64

In [372]:
# get the top ten damsires represented over all years (2002 - 2023)

top_damsires = horse_df['Damsire'].value_counts().head(10)

print(top_damsires)

Rubinstein          25
De Niro             18
Sandro Hit          17
Jazz                16
Rotspon             16
Weltmeyer           14
Ferro               13
Krack C             11
Sir Donnerhall I    11
Rohdiamant          11
Name: Damsire, dtype: int64

In [374]:
# get the top ten damsires of Grand Prix horses from 2002-2023

gp_damsires = horse_df.loc[(horse_df["HighestLevel"] == "Grand Prix")]
gp_damsires = gp_damsires['Damsire'].value_counts().head(10)

print(gp_damsires)

De Niro           7
Rubinstein        6
Rotspon           6
Brentano II       5
Ferro             5
Weltmeyer         4
Rohdiamant        4
Jazz              4
Flemmingh         3
Wolkenstein II    3
Name: Damsire, dtype: int64

In [364]:
# get the top ten most prominent breeders represented over all years (2002 - 2023)

top_breeders = horse_df['Breeder'].value_counts().head(10)

print(top_breeders)

DG Bar Ranch          16
Maryanna Haymon       12
Nancy Holowesko        9
Leatherdale Farms      7
Gestut Lewitz          6
Oak Hill Ranch         6
Judy Yancey            6
Horses Unlimited       6
Jackie Ahl-Eckhaus     5
Maurine Swanson        5
Name: Breeder, dtype: int64

In [366]:
# get the top ten countries that horses were bred in over all years (2002 - 2023)

top_countries = horse_df['Country'].value_counts().head(10)

print(top_countries)

USA              307
Germany          299
Netherlands      165
Denmark           26
Canada             7
Belgium            7
Spain              5
Great Britain      5
Norway             4
Sweden             2
Name: Country, dtype: int64

In [40]:
# get the top ten studbooks represented over all years (2002 - 2023)

top_studbooks = horse_df['Studbook'].value_counts().head(10)

print(top_studbooks)

KWPN                  243
Hanoverian            241
Oldenburg             180
Westfalen              48
Danish Warmblood       34
Rhinelander            18
American Warmblood     10
PRE                     7
Holsteiner              7
Swedish Warmblood       6
Name: Studbook, dtype: int64


In [368]:
# get the top horses with the most championship appearances over all years (2002 - 2023)

most_appearances = championship_df['Horse'].value_counts().head(10)

print(most_appearances)

WakeUp                  6
Quantum Jazz            5
Flavius MF              5
Sole Mio                5
Sternlicht Hilltop      5
Floretienne             5
Pikko del Cerro HU      5
Fashion Designer OLD    5
Don Cesar               5
Au Revoir               5
Name: Horse, dtype: int64